## Import Libraries

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
from tqdm import tqdm
from joblib import parallel_backend
from Bio import SeqIO
from sklearn.base import BaseEstimator, TransformerMixin
from itertools import permutations
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display= 'diagram')
from sklearn.metrics import classification_report, plot_roc_curve, plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC
from src.KMerTransformers import *

## Import DNA Sequence Data

I will be using the BioPython library to parse through fasta files found on the RefSeq database.

Each file contains sequences from the genome of a species. They are each tens of thousands of rows long, but I will limit each dataframe to 20000 rows per species.

Let's look at one record as an example of the data that is being parsed in.

In [ ]:
for seq_record in SeqIO.parse("data/GCF_000001405.40_GRCh38.p14_cds_from_genomic.fna", "fasta"): #human fasta file
    print(seq_record)
    break #break after printing one record

This project will be using using the sequence (Seq) as data in the model, but let's define a helper function that will collect a few more columns of data, just in case I want to reference them later.

In [ ]:
def parse_from_genome_file(filename, target_name, n=None):
    
    i=0
    seqs = []
    genes = []
    proteins = []
    target = []
    
    for seq_record in SeqIO.parse(filename, "fasta"):
        
        seqs.append(''.join(seq_record.seq)) #append the sequence data
        genes.append(seq_record.description.split('[')[1][5:-2]) #append gene name
        proteins.append(seq_record.description.split('[')[3][8:-2]) #append protein description
        target.append(target_name) #append the class name ('human' or otherwise)
        
        if n != None:
            if i < n:
                i+=1
            if i >= n:
                break
            
    df = pd.concat([pd.DataFrame(seqs),pd.DataFrame(genes), pd.DataFrame(proteins), pd.DataFrame(target)], axis=1)
    df.columns = ['seq','gene','protein', 'target']
    
    return df

In [ ]:
#create dataframes for each genome file
human_df = parse_from_genome_file("data/GCF_000001405.40_GRCh38.p14_cds_from_genomic.fna", 'human', n=20000)
chimp_df = parse_from_genome_file('data/GCF_002880755.1_Clint_PTRv2_cds_from_genomic.fna', 'chimp', n=20000)
dolphin_df = parse_from_genome_file('data/GCF_011762595.1_mTurTru1.mat.Y_cds_from_genomic.fna', 'dolphin', n=20000)
oak_df = parse_from_genome_file('data/GCF_001633185.2_ValleyOak3.2_cds_from_genomic.fna', 'oak', n=20000)
mushroom_df = parse_from_genome_file('data/GCF_017499595.1_MGC_Penvy_1_cds_from_genomic.fna', 'mushroom', n=20000)


In [ ]:
human_df.head()

In [ ]:
human_df.loc[0,'seq']

## EDA

Before I do any analysis, I will add all of the species from the DNA sequence data I collected into one dataframe.

In [ ]:
species_compare_df = pd.concat([human_df, chimp_df, dolphin_df, oak_df, mushroom_df], axis=0)

In [ ]:
species_compare_df.head()

In [ ]:
species_compare_df.info()

The first thing we will want to see are the metrics of the DNA sequence itself. Let's look at the length of the sequence, grouped by species.

In [ ]:
species_compare_df['seq_length'] = species_compare_df.seq.apply(lambda x: len(x))
print('DNA Sequence Length Info by Species')
species_compare_df.groupby('target').seq_length.describe().T

We can now determine that the mean length of each sequence between targets is relatively the same, apart from the Oak Tree and Mushroom genomic data.

Using the custom KMerTransformer function that I imported, I can transform the data and do some basic visual analysis. I will start with k=3 so I don't have too many columns and I can see the data together easily.

In [ ]:
kmer_transformer = KMerTransformer(k=3, verbose=False)
kmer_matrix = kmer_transformer.fit_transform(species_compare_df)
kmer_matrix = pd.concat([kmer_matrix, species_compare_df.target], axis=1)

Let's quickly inspect that the data is populating in the columns correctly.

In [ ]:
kmer_matrix.head()

5 rows × 65 columns

In [ ]:
#plot a heatmap of correlation between features
kmer_corr = kmer_matrix.corr()
fig = plt.figure(figsize=(10,8))
plt.title('Correlation between trimers', fontdict={'fontsize': 20})
sns.set(font_scale=.6)
sns.heatmap(kmer_corr, cmap="vlag")
plt.tight_layout()
plt.show()

This heatmap shows that there is high correlation between most of our features. This will affect our model, and I may be able to find a way to extract features that have no correlation in common, or classifiers that can deal with a high levele of colinearity.

**Mean Values for each Trimer Grouped by Species**

Now, let's take a look at the mean values for every column group by the 4 species' DNA I added to this dataframe as an example.

5 rows × 64 columns

In [ ]:
#set up the dimension of each subplot and the final graph
rows = 16
cols = 4
fig_cols = 5
fig_rows = 1

#get the total max and min for scaling later
mean_agg = mean_agg_df
mean_max = mean_agg.max().max()
mean_min = mean_agg.min().min()

#Extract the title for each subplot, and the trimers in each matrix
mean_agg_index = [f'{x.title()} Genome' for x in mean_agg.index]
kmer_index = np.array([x[0] for x in mean_agg.columns]).reshape(rows, cols)
#scale the data with MinMax
mean_agg = (np.array(mean_agg) - mean_min) / (mean_max - mean_min)

#plot the graph
fig = make_subplots(rows=fig_rows, cols=fig_cols, subplot_titles=mean_agg_index)
layout = {'title':'Scaled Mean Values of Each Trimer Grouped by Genome', 'width':1200, 'height':600}
for row_index in range(fig_rows):
    for col_index in range(fig_cols):
        index = col_index + fig_rows*row_index
        array = np.array(mean_agg[index]).astype(float).reshape(rows, cols)
        #draw a heatmap for genome
        fig.add_trace(go.Heatmap(z=array, coloraxis='coloraxis'), row=row_index+1, col=col_index+1)
        #draw each trimer over each square in each subplot
        for k in range(rows):
            for j in range(cols):
                fig.add_annotation(text=np.array(kmer_index).reshape(rows, cols)[k][j], 
                                   x=j, y=k, showarrow=False, row=row_index+1, col=col_index+1, font=dict(size=9, color="#222299"))
        layout[f'xaxis{index+1 if index > 0 else ""}'] = dict(visible=False)
        layout[f'yaxis{index+1 if index > 0 else ""}'] = dict(autorange='reversed',visible=False)

fig.update_layout(layout)
fig.update_coloraxes(colorscale = 'bupu')
plt.show()

In this graph, we are visualizing those mean values for every columns arranged as a 2-dimensional grid. Each subplot is one of the four species, and now we can visually compare the means of their values together. It looks like there are some patterns emerging already. For the mammalian genomes (human, chimp, and dolphin) there are a lot of areas that are visually similiar. Even some parts of the Oak genome are related to the three others, but for the most part it is easy to distinguish already that it is different.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similiarity_map = pd.DataFrame(cosine_similarity(np.array(mean_agg_df),np.array(mean_agg_df)), index=mean_agg_df.index, columns=mean_agg_df.index)
fig = plt.figure(figsize=(8,5))
sns.set(font_scale=1)
sns.set_style("whitegrid")
sns.heatmap(cosine_similiarity_map)
plt.suptitle('Cosine Similarity between Genomes (Mean Values)')
plt.show()

We have visually seen the similiarity between our features in different classes, but I also compute the cosine similiarity between each class. Note that the bottom of this scale is still a value of high similiarity. It seems that all of our classes have a high value of cosine similiarity.

Now let's take a look at a visualization of the variance in each genome's features.

In [ ]:
var_agg_df = kmer_matrix.groupby('target', sort=False).agg(['std'])
var_agg_df

In [ ]:
#set up the dimension of each subplot and the final graph
rows = 16
cols = 4
fig_cols = 5
fig_rows = 1

#Extract the title for each subplot, and the trimers in each matrix
var_agg = var_agg_df
var_agg_index = [f'{x.title()} Genome' for x in var_agg.index]
kmer_index = np.array([x[0] for x in var_agg.columns]).reshape(rows, cols)
var_agg = np.array(var_agg)
           
#plot the graph
fig = make_subplots(rows=fig_rows, cols=fig_cols, subplot_titles=var_agg_index)
layout = {'title':'Scaled Variance of Each Trimer Column Grouped by Genome', 'width':1200, 'height':600}
for row_index in range(fig_rows):
    for col_index in range(fig_cols):
        index = col_index + fig_rows*row_index
        array = np.array(var_agg[index]).astype(float).reshape(rows, cols)
        #draw a heatmap for genome
        fig.add_trace(go.Heatmap(z=array, coloraxis='coloraxis'), row=row_index+1, col=col_index+1)
        #draw each trimer over each square in each subplot
        for k in range(rows):
            for j in range(cols):
                fig.add_annotation(text=np.array(kmer_index).reshape(rows, cols)[k][j], 
                                   x=j, y=k, showarrow=False, row=row_index+1, col=col_index+1, font=dict(size=9, color="#222299"))
        layout[f'xaxis{index+1 if index > 0 else ""}'] = dict(visible=False)
        layout[f'yaxis{index+1 if index > 0 else ""}'] = dict(autorange='reversed',visible=False)

fig.update_layout(layout)
fig.update_coloraxes(colorscale = 'tempo')
plt.show()

Based on this visualization, we can see that some genomes have higher variance than others. It will be interesting to see if this has an impact on multiclass models, or could be explained during binary classification of two classes that have different levels of variance.

## Train Test Split

I will want to do a multi-class model later on, but I will start with a 2 class dataframe to create a baseline model.

In [ ]:
baseline_df = pd.concat([human_df, oak_df], axis=0).reset_index(drop=True)

In [ ]:
labeler = LabelEncoder()

X = baseline_df.drop(columns='target')
y = labeler.fit_transform(baseline_df['target'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Now that I have split the data into training and testing sets, I can transform the X's into the dataframe's we will use in the model using the custom KMerTransformer that was imported. I will start the baseline model with k=3.

In [ ]:
kmer_transformer = KMerTransformer(k=3, verbose=False)
X_train_trans = kmer_transformer.fit_transform(X_train)
X_test_trans = kmer_transformer.transform(X_test)

## Baseline Model for Binary Classification

In [ ]:
baseline_model = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42))
], verbose=True)
baseline_model

In [ ]:
baseline_model.fit(X_train_trans, y_train)
y_preds = baseline_model.predict(X_test_trans)

In [ ]:
print(classification_report(labeler.inverse_transform(y_test), labeler.inverse_transform(y_preds)))

plot_confusion_matrix(baseline_model, X_test_trans, y_test)
plt.show()

plot_roc_curve(baseline_model, X_test_trans, y_test)
plt.show()

## Human vs Chimpanzee DNA Binary Classifier

We can assume that Human and Chimpanzee DNA is much more similar than Human and Oak DNA from our intial EDA and cosine similiarity tests. I will try to run the same LogisiticRegression classifier on just Human and Chimapanzee DNA

In [ ]:
human_chimp_df = pd.concat([human_df, chimp_df], axis=0).reset_index(drop=True)

labeler = LabelEncoder()

X = human_chimp_df.drop(columns='target')
y = labeler.fit_transform(human_chimp_df['target'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

kmer_transformer = KMerTransformer(k=3, verbose=False)
X_train_trans = kmer_transformer.fit_transform(X_train)
X_test_trans = kmer_transformer.transform(X_test)

human_chimp_model = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42))
], verbose=True)

human_chimp_model.fit(X_train_trans, y_train)
y_preds = human_chimp_model.predict(X_test_trans)

print(classification_report(labeler.inverse_transform(y_test), labeler.inverse_transform(y_preds)))

In [ ]:
plot_confusion_matrix(human_chimp_model, X_test_trans, y_test)
plt.suptitle('Human DNA vs Chimpanzee DNA Logisitic Regression Confusion Matrix')

plt.show()

In [ ]:
plot_roc_curve(human_chimp_model, X_test_trans, y_test)
plt.show()

The untuned Logisitic Regression is not accurate at all. A 53% accuracy score is barely more than just random chance because the two classes in the test set are balanced. This may be due to the high colinearity of our features. Since I am sticking with this feature set for now, I will try out different classifiers that may work better with high colinearity.

## Grid Search for Best Binary Classifier, k=3

Since Logisitic Regression does not seem to be a good classifier for our data, I will implement a grid search on different parameters for many different classifiers to see if we can find one that will fit the data better. The model is still using the same training and testing data trimers (k=3) from the KMerTransformer. I will assume that with a higher k, we will see a better accuracy score, but it will also take longer to fit each model. If this grid search finds good classifiers, I could increase the k in another grid search, but on a smaller set of parameters. First let's just try the untuned classifiers.

In [ ]:
num_features = len(X_train_trans.columns)

human_chimp_model = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42))
])

RANDOM_STATE = 42

params = [
    {'classifier':[LogisticRegression(random_state=RANDOM_STATE)]},
    {'classifier':[GaussianNB()]},
    {'classifier':[KNeighborsClassifier()]},
    {'classifier':[RandomForestClassifier(random_state=RANDOM_STATE)]},
    {'classifier':[ExtraTreesClassifier(random_state=RANDOM_STATE)]},
    {'classifier':[XGBClassifier(random_state=RANDOM_STATE)]},
    {'classifier':[GradientBoostingClassifier(random_state=RANDOM_STATE)]},
    {'classifier':[SVC(random_state=RANDOM_STATE)]}
]

grid_searcher_clf = GridSearchCV(human_chimp_model, params, cv=3, scoring='accuracy', verbose=1)

with parallel_backend('threading', n_jobs=-1):
    grid_searcher_clf.fit(X_train_trans, y_train)
    best_est = grid_searcher_clf.best_estimator_
    y_preds = best_est.predict(X_test_trans)
    
print('Best Estimator:')
print(best_est)

print(classification_report(labeler.inverse_transform(y_test), labeler.inverse_transform(y_preds)))

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
sns.set(font_scale = .65)
cv_results = list(grid_searcher_clf.cv_results_['mean_test_score'])
cv_params = [re.sub(r'\(.*?\)', "", str(x['classifier']).replace("\n", "")) for x in grid_searcher_clf.cv_results_['params']]
sns.barplot(cv_params, cv_results)
fig.suptitle('Accuracy of Different Classifiers in GridSearchCV ')
plt.show()

With the baseline scores for each untuned classifier, ExtraTreesClassifier had the highest accuracy so we will continue to tune with that model. First, let's look at the accuracy and metrics for an untuned ExtraTreesClassifier when k=6.

In [ ]:
human_chimp_model = Pipeline(steps=[
    ('kmer_transformer', KMerTransformer(k=6, verbose=False)),
    ('scaler', StandardScaler()),
    ('classifier', ExtraTreesClassifier())
])

human_chimp_model.fit(X_train, y_train)
y_preds = human_chimp_model.predict(X_test)

print(classification_report(labeler.inverse_transform(y_test), labeler.inverse_transform(y_preds)))

In [ ]:
sns.set(font_scale = 1)
plot_confusion_matrix(human_chimp_model, X_test, y_test)
plt.suptitle('Human DNA vs Chimpanzee DNA ExtraTrees Confusion Matrix')
plt.savefig('img/graphs/untuned_extratrees_confusion.png', dpi=300)
plt.show()

In [ ]:
plot_roc_curve(human_chimp_model, X_test, y_test)
plt.suptitle('Human DNA vs Chimpanzee DNA ROC Curve')
plt.savefig('img/graphs/untuned_extratrees_roc.png', dpi=300)
plt.show()

## K-Group K-mer Transformer

Here I will create another custom transformer for preprocessing the data. Similar to the KMerTransformer, this transformer will create frequency dictionaries for each row of data. Unlike the KMerTransformer, it will pool all the A's, C's, G's, and T's counts' instead of their unique sequences.

In other words, if we have a sequence like 'AAGTCGAGT' we will have 3 A's, 1 C, 3 G's, and 2 T's. So the sequence will populate a column named: 'A2C1G3T2'. Similiarly, a sequence like 'GTGAAACTG' would also populate the same column. This means that we will create many times less columns for the same k. Thus, we will increase k significantly when we call the transformer.

With the imported KGroupKMerTransformer class, let's try it out on the data with a k=12

In [ ]:
human_chimp_df = pd.concat([human_df, chimp_df], axis=0).reset_index(drop=True)

labeler = LabelEncoder()

X = human_chimp_df.drop(columns='target')
y = labeler.fit_transform(human_chimp_df['target'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

human_chimp_model_k12group = Pipeline(steps=[
    ('kgroup_transformer', KGroupKMerTransformer(k=12, verbose=False)),
    ('kgroup_column_pruner', KGroupColumnPruner()),
    ('scaler', StandardScaler()),
    ('classifier', ExtraTreesClassifier(random_state=42))
], verbose=True)

human_chimp_model_k12group.fit(X_train, y_train)
y_preds = human_chimp_model_k12group.predict(X_test)

print(classification_report(labeler.inverse_transform(y_test), labeler.inverse_transform(y_preds)))

As you can see, we get 86% accuracy for a fraction of the columns we used in the KMerTransformer when k=6. Let's continue this experiment and increase k to 36

In [ ]:
human_chimp_model_k36group = Pipeline(steps=[
    ('kgroup_transformer', KGroupKMerTransformer(k=36, verbose=False)),
    ('kgroup_column_pruner', KGroupColumnPruner()),
    ('scaler', StandardScaler()),
    ('classifier', ExtraTreesClassifier(random_state=42))
], verbose=True)

human_chimp_model_k36group.fit(X_train, y_train)
y_preds = human_chimp_model_k36group.predict(X_test)

print(classification_report(labeler.inverse_transform(y_test), labeler.inverse_transform(y_preds)))

In [ ]:
fig, ax = plt.subplots(figsize=(15,5), ncols=2)
sns.set(font_scale = 1)
plot_confusion_matrix(human_chimp_model_k12group, X_test, y_test, ax=ax[0])
plot_confusion_matrix(human_chimp_model_k36group, X_test, y_test, ax=ax[1])
ax[0].set_title('KGroups Confusion Matrix (k=12)')
ax[1].set_title('KGroups Confusion Matrix (k=36)')
plt.show()

In [ ]:
fig = plt.figure()
plot_roc_curve(human_chimp_model_k12group, X_test, y_test)
plt.suptitle('KGroups ROC-AUC (k=12)')
plt.show()

In [ ]:
plot_roc_curve(human_chimp_model_k36group, X_test, y_test)
plt.suptitle('KGroups ROC-AUC (k=36)')
plt.show()

Now we get 90% accuracy when comparing Human and Chimpanzee DNA and k=36. After pruning columns that were never populated, we used 7386 total columns. Our AUC score is now 96% and is looking even better than in our previous model.

So far, so good - let's try this model on Multi-Class data.

## Multi-Class Classification

Now we will try the KGroupKMerTransformer on Multiclass data with k=36.

In [ ]:
species_compare_df = pd.concat([human_df, chimp_df, dolphin_df, oak_df, mushroom_df], axis=0)

multilabeler = LabelEncoder()

X = species_compare_df.drop(columns='target')
y = multilabeler.fit_transform(species_compare_df['target'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

multiclass_kgroup_model = Pipeline(steps=[
    ('kgroup_transformer', KGroupKMerTransformer(k=36, verbose=False)),
    ('kgroup_column_pruner', KGroupColumnPruner()),
    ('scaler', StandardScaler()),
    ('classifier', ExtraTreesClassifier(random_state=42))
], verbose=True)

multiclass_kgroup_model.fit(X_train, y_train)
y_preds = multiclass_kgroup_model.predict(X_test)

print('Multiclass Performance with KGroup Transformer (k=36)')
print(classification_report(multilabeler.inverse_transform(y_test), multilabeler.inverse_transform(y_preds)))

It seems that the KGroup Multiclass model is a little less accurate than the binary model. Especially in certain classes. For example, the f1-score for Mushroom is particularily low compared to the others. Let's run the original KMerTransformer model on Multiclass data and compare the results.

In [ ]:
multiclass_kmer_model = Pipeline(steps=[
    ('kmer_transformer', KMerTransformer(k=6, verbose=False)),
    ('scaler', StandardScaler()),
    ('classifier', ExtraTreesClassifier(random_state=42))
], verbose=True)

multiclass_kmer_model.fit(X_train, y_train)
y_preds = multiclass_kmer_model.predict(X_test)

print('Multiclass Performance with KMer Transformer (k=6)')
print(classification_report(multilabeler.inverse_transform(y_test), multilabeler.inverse_transform(y_preds)))

In [ ]:
fig, ax = plt.subplots(figsize=(15,5), ncols=2)
plot_confusion_matrix(multiclass_kmer_model, X_test, y_test, ax=ax[1])
plot_confusion_matrix(multiclass_kgroup_model, X_test, y_test, ax=ax[0])
ax[0].set_yticklabels(labels=multilabeler.classes_)
ax[1].set_yticklabels(labels=multilabeler.classes_)
ax[0].set_xticklabels(labels=multilabeler.classes_, rotation=45)
ax[1].set_xticklabels(labels=multilabeler.classes_, rotation=45)
ax[0].set_title('K-Groups Transformer (k=36) - Accuracy 87%')
ax[1].set_title('K-Mer Transformer (k=6) - Accuracy 89%')
plt.show()

We can see in this comparison of confusion matrices that the KMerTransformer preprocessed model is much better at classifying some parts of the data that the KGroup Model has a harder time with. Let's try to combine both models into one, and compare those results.

## KGroup + KMer Model

In this section, we will combine the two previous preprocessing transformers we created into one model. First, we will determine the best k for KGroups if KMer k=6.

After that, we can find the best hyperparameters for the ExtraTreesClassifier using the k we find in the first grid search.

**Finding Best K for KGroup**

In [ ]:
species_compare_df = pd.concat([human_df, chimp_df, dolphin_df, oak_df, mushroom_df], axis=0)

species_compare_df_sample = species_compare_df.sample(n=10000, random_state=42)

multilabeler = LabelEncoder()

X = species_compare_df_sample.drop(columns='target')
y = multilabeler.fit_transform(species_compare_df_sample['target'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

multiclass_model = Pipeline(steps=[
    ('trans_union', FeatureUnion([
        ('p1', Pipeline([
            ('kgroup_transformer', KGroupKMerTransformer(k=12, verbose=False)),
            ('kgroup_column_pruner', KGroupColumnPruner(verbose=False)),
        ])),
        ('p2', Pipeline([
            ('kmer_transformer', KMerTransformer(k=6,verbose=False))
        ])),
    ])),    
    ('scaler', StandardScaler()),
    ('classifier', ExtraTreesClassifier(random_state=42))
])


params = {
    'trans_union__p1__kgroup_transformer__k':[6,12,24,36],
}

grid_searcher = GridSearchCV(multiclass_model, param_grid=params, cv=3, scoring='accuracy', verbose=2)

grid_searcher.fit(X_train, y_train)

In [ ]:
best_est = grid_searcher.best_estimator_
y_preds = best_est.predict(X_test)
    
print('Best Estimator:')
print(best_est)

In [ ]:
grid_searcher.cv_results_

Based on these results. The best k for KGroup Transformer is k=24, when the KMerTransformer k=6. Let's try that in next section, and fine tune the ExtraTreesClassifier using a grid search.

**Finding best hyperparameters for ExtraTreesClassifier**

In [ ]:
species_compare_df = pd.concat([human_df, chimp_df, dolphin_df, oak_df, mushroom_df], axis=0)

species_compare_df_sample = species_compare_df.sample(n=10000, random_state=42)

multilabeler = LabelEncoder()

X = species_compare_df_sample.drop(columns='target')
y = multilabeler.fit_transform(species_compare_df_sample['target'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

k_trans = Pipeline([
    ('trans_union', FeatureUnion([
            ('p1', Pipeline([
                ('kgroup_transformer', KGroupKMerTransformer(k=24, verbose=False)),
                ('kgroup_column_pruner', KGroupColumnPruner(verbose=False)),
            ])),
            ('p2', Pipeline([
                ('kmer_transformer', KMerTransformer(k=6, verbose=False))
            ]))
        ]))
])

X_train_trans = k_trans.fit_transform(X_train)
X_test_trans = k_trans.transform(X_test)

multiclass_model = Pipeline(steps=[  
    ('scaler', StandardScaler()),
    ('classifier', ExtraTreesClassifier(random_state=42))
])


params = {
    'classifier__n_estimators':range(100,350,50),
    'classifier__criterion':['gini', 'entropy', 'log_loss'],
    'classifier__max_features':['sqrt','log2'],
    'classifier__min_samples_leaf':np.linspace(1,50,8, dtype=int)
}

grid_searcher = GridSearchCV(multiclass_model, param_grid=params, cv=3, scoring='accuracy', verbose=1)

with parallel_backend('threading', n_jobs=-1):
    grid_searcher.fit(X_train_trans, y_train)

In [ ]:
grid_searcher.best_estimator_

## Final Model

For the final model, I will add the hyperparameters I found from the grid searches.

- KGroupTransformer: k=24
- KMerTransformer: k=6
- ExtraTreesClassifier: n_estimators=300

In [ ]:
X = species_compare_df.drop(columns='target')
y = multilabeler.fit_transform(species_compare_df['target'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

final_model = Pipeline(steps=[
    ('trans_union', FeatureUnion([
        ('p1', Pipeline([
            ('kgroup_transformer', KGroupKMerTransformer(k=24, verbose=False)),
            ('kgroup_column_pruner', KGroupColumnPruner(verbose=False)),
        ])),
        ('p2', Pipeline([
            ('kmer_transformer', KMerTransformer(k=6, verbose=False))
        ])),
    ])),    
    ('scaler', StandardScaler()),
    ('classifier', ExtraTreesClassifier(n_estimators=300, random_state=42))
])

final_model.fit(X_train, y_train)

y_preds = final_model.predict(X_test)

final_model

In [ ]:
print(classification_report(multilabeler.inverse_transform(y_test), multilabeler.inverse_transform(y_preds)))

from yellowbrick.classifier import ClassificationReport

visualizer = ClassificationReport(final_model, classes=multilabeler.classes_, support=True)

visualizer.fit(X_train, y_train)        # Fit the visualizer and the model
visualizer.score(X_test, y_test)        # Evaluate the model on the test data
visualizer.show(outpath="img/graphs/final_model_classification")

In [ ]:
from yellowbrick.classifier import ClassPredictionError

visualizer = ClassPredictionError(final_model, classes=multilabeler.classes_)

visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.show(outpath="img/graphs/final_model_clf_error")

In [ ]:
plot_confusion_matrix(final_model, X_test, y_test)
plt.yticks(ticks=range(len(multilabeler.classes_)), labels=multilabeler.classes_)
plt.xticks(ticks=range(len(multilabeler.classes_)), labels=multilabeler.classes_, rotation=45)
plt.suptitle('Multiclass ExtraTreesClassifier - KMer + KGroup Model ')
plt.savefig('img/graphs/final_model_confusion.png', dpi=300)
plt.show()

In [ ]:
from yellowbrick.classifier import ROCAUC

visualizer = ROCAUC(final_model, classes=multilabeler.classes_)

visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.show(outpath="img/graphs/final_model_roc_auc") 

## Results
My final model achieved a 90% acccuracy on test data for classifying sequences between 5 genomes of DNA. Although there is room for improvement in the model, as well as trying out other classification methods (i.e. neural networks), I can reject my null hypothesis, and we can assume there are patterns of similiarity between the classes of the genomic data. 

Moreover, we are able to distinguish the patterns and variance and classify species from individual DNA sequences with high accuracy.